In [2]:
from os import chdir
chdir('/home/jovyan')

In [3]:
from lib import postgres as pg

# 1. How to deal with de missing values?

Replace the missing data with values from exact 7 days ago, otherwise replace with values from 7 days in the future.
- The power consumption of an individual household is very dependent on time variables such as **hour of the day** and **day of the week**.
- **1 Day = 1440 Timesteps**

In [3]:
data = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    present.datetime,
    coalesce(present.global_active_power, past.global_active_power, future.global_active_power)  as global_active_power,
    coalesce(present.global_reactive_power, past.global_reactive_power, future.global_reactive_power)  as global_reactive_power,
    coalesce(present.voltage, past.voltage, future.voltage)  as voltage,
    coalesce(present.current, past.current, future.current)  as current,
    coalesce(present.sub_metering_1, past.sub_metering_1, future.sub_metering_1)  as sub_metering_1,
    coalesce(present.sub_metering_2, past.sub_metering_2, future.sub_metering_2)  as sub_metering_2,
    coalesce(present.sub_metering_3, past.sub_metering_3, future.sub_metering_3)  as sub_metering_3
FROM 
    (
        SELECT 
            * 
        FROM 
            staging.individual_household_power_consumption 
        WHERE 
            global_active_power is null 
        ORDER BY 1 ASC 
        LIMIT 10
    ) present
LEFT JOIN
    staging.individual_household_power_consumption past
ON 
    past._id = present._id - 7*1440 
LEFT JOIN
    staging.individual_household_power_consumption future
ON
    future._id = present._id + 7*1440
) main
ORDER BY 1 ASC
LIMIT 10
""")
data.head()

,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3
0,2006-12-21 11:23:00,0.332,0.192,244.66,1.6,0.0,0.0,0.0
1,2006-12-21 11:24:00,0.328,0.188,243.92,1.6,0.0,0.0,0.0
2,2006-12-30 10:08:00,2.730,0.062,240.21,11.2,0.0,0.0,0.0
3,2006-12-30 10:09:00,2.718,0.060,239.38,11.2,0.0,0.0,0.0
4,2007-01-14 18:36:00,4.934,0.340,230.63,22.2,0.0,29.0,16.0


In [4]:
cmd = """
CREATE SCHEMA clean;
CREATE TABLE clean.individual_household_power_consumption (
    _id SERIAL PRIMARY KEY,
    datetime TIMESTAMP,
    global_active_power FLOAT,
    global_reactive_power FLOAT,
    voltage FLOAT,
    current FLOAT,
    sub_metering_1 FLOAT,
    sub_metering_2 FLOAT,
    sub_metering_3 FLOAT
);
"""

pg.run_command(cmd)

None


#### **Notes**:

1. (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.
    - kW -> W.h : 60/1000

In [5]:
data = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    present.datetime,
    coalesce(present.global_active_power, past.global_active_power, future.global_active_power)  as global_active_power,
    coalesce(present.global_reactive_power, past.global_reactive_power, future.global_reactive_power)  as global_reactive_power,
    coalesce(present.voltage, past.voltage, future.voltage)  as voltage,
    coalesce(present.current, past.current, future.current)  as current,
    60*coalesce(present.sub_metering_1, past.sub_metering_1, future.sub_metering_1)/1000  as sub_metering_1,
    60*coalesce(present.sub_metering_2, past.sub_metering_2, future.sub_metering_2)/1000  as sub_metering_2,
    60*coalesce(present.sub_metering_3, past.sub_metering_3, future.sub_metering_3)/1000  as sub_metering_3
FROM 
    staging.individual_household_power_consumption present
LEFT JOIN
    staging.individual_household_power_consumption past
ON 
    past._id = present._id - 7*1440 
LEFT JOIN
    staging.individual_household_power_consumption future
ON
    future._id = present._id + 7*1440
) main
ORDER BY 1 ASC
""")
data.head()

,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,0.06,1.02
1,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,0.06,0.96
2,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,0.12,1.02
3,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,0.06,1.02
4,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,0.06,1.02


In [6]:
pg.bulk_load_df(data, 'clean', 'individual_household_power_consumption')

Connecting to Database
Starting DataFrame CSV export...
CSV File has been created
The table clean.individual_household_power_consumption has been successfully truncated.
The incremental _id for table clean.individual_household_power_consumption has been reset.
The data has been succesfully loaded to table clean.individual_household_power_consumption
DB connection closed.
Removing temporary files...
Done.
Elapsed time: 28.555450439453125 seconds


In [8]:
check_target = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    *
FROM 
    clean.individual_household_power_consumption
) main
ORDER BY 1 ASC
LIMIT 10;
""")
check_target.head()

,_id,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3
0,1,2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,0.06,1.02
1,2,2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,0.06,0.96
2,3,2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,0.12,1.02
3,4,2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,0.06,1.02
4,5,2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,0.06,1.02


# 2. Resampling

The original [**IHPC**](https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption#) dataset is minute-grained, but the load forecasting problem might involve the forecast for different horizons - minutes, hours, days, depending on its application on the energy industry. Therefore, it is necessary to resample the dataset for different time granularities. In this project we will be exploring the minute and hourly-grained variations of the **IHPC** dataset.

The function used on the resampling of the dataset will be the **average**. The values will also be truncated with **3 decimal digits**.

In [14]:
resampled_data = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    date_trunc('hour', present.datetime) as datetime,
    ROUND(CAST(AVG(present.global_active_power) AS NUMERIC),3) as global_active_power,
    ROUND(CAST(AVG(present.global_reactive_power) AS NUMERIC),3) as global_reactive_power,
    ROUND(CAST(AVG(present.voltage) AS NUMERIC),3) as voltage,
    ROUND(CAST(AVG(present.current) AS NUMERIC),3) as current,
    ROUND(CAST(AVG(present.sub_metering_1) AS NUMERIC),3) as sub_metering_1,
    ROUND(CAST(AVG(present.sub_metering_2) AS NUMERIC),3) as sub_metering_2,
    ROUND(CAST(AVG(present.sub_metering_3) AS NUMERIC),3) as sub_metering_3
FROM 
    clean.individual_household_power_consumption present
GROUP BY 1
) main
ORDER BY 1 ASC;
""")
resampled_data.head()

,datetime,global_active_power,global_reactive_power,voltage,current,sub_metering_1,sub_metering_2,sub_metering_3
0,2006-12-16 17:00:00,4.223,0.229,234.644,18.100,0.0,0.032,1.012
1,2006-12-16 18:00:00,3.632,0.080,234.580,15.600,0.0,0.403,1.012
2,2006-12-16 19:00:00,3.400,0.085,233.233,14.503,0.0,0.086,1.001
3,2006-12-16 20:00:00,3.269,0.075,234.072,13.917,0.0,0.000,1.007
4,2006-12-16 21:00:00,3.056,0.077,237.159,13.047,0.0,0.025,1.033


In [13]:
cmd = """
CREATE TABLE clean.individual_household_power_consumption_h (
    _id SERIAL PRIMARY KEY,
    datetime TIMESTAMP,
    global_active_power FLOAT,
    global_reactive_power FLOAT,
    voltage FLOAT,
    current FLOAT,
    sub_metering_1 FLOAT,
    sub_metering_2 FLOAT,
    sub_metering_3 FLOAT
);
"""

pg.run_command(cmd)

None


In [15]:
pg.bulk_load_df(resampled_data, 'clean', 'individual_household_power_consumption_h')

Connecting to Database
Starting DataFrame CSV export...
CSV File has been created
The table clean.individual_household_power_consumption_h has been successfully truncated.
The incremental _id for table clean.individual_household_power_consumption_h has been reset.
The data has been succesfully loaded to table clean.individual_household_power_consumption_h
DB connection closed.
Removing temporary files...
Done.
Elapsed time: 0.5478858947753906 seconds


In [17]:
check_target = pg.load_query_to_df("""
SELECT main.* FROM
(
SELECT
    count(*) as count
FROM 
    clean.individual_household_power_consumption_h
) main
ORDER BY 1 ASC
LIMIT 10;
""")
check_target.head()

,count
0,34589
